In [11]:
from data_preprocessing import *

In [22]:
adata = anndata.read_h5ad('../results/concat.h5ad')

adata.obs

,batch,cell_type
AAACAGCCAAACGCGA-1-0,D8_1,Somite
AAACAGCCACAGGAAT-1-0,D8_1,Somite
AAACAGCCACATGCTA-1-0,D8_1,Somite
AAACAGCCAGGCCATT-1-0,D8_1,Somite
AAACAGCCATCCGTAA-1-0,D8_1,Somite
...,...,...
TTTGTGTTCCCGTTGT-1-7,D22_2,Immature neuron
TTTGTGTTCGTAATCA-1-7,D22_2,Mesenchymal
TTTGTTGGTCCGCTGT-1-7,D22_2,Mesenchymal
TTTGTTGGTGTGAGAG-1-7,D22_2,Somite


In [67]:
import pyBigWig

bw = pyBigWig.open('../results/bam_cell_type/D8_1/Somite.bw')
bw.values("10", 100004794, 100005512)

[6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0,
 13.0

In [12]:
dataset = SequenceDataset('../results/concat.h5ad','../data/hg38.fa')

/Users/adorman/.local/lib/python3.11/site-packages/pandas/core/generic.py:6326: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  self[name] = value


In [ ]:
dataset.sequence_ATAC

,peakID,ATAC,cell_type,dataset
0,10:100004794-100005512,0.0,12,D8_1
1,10:100004820-100005525,0.0,12,D8_1
2,10:100004829-100005566,0.0,12,D8_1
3,10:100005947-100006870,0.0,12,D8_1
4,10:100005972-100006890,0.0,12,D8_1
...,...,...,...,...
109793182,Y:9320873-9321803,0.0,Somite,D22_2
109793183,Y:9320878-9321792,0.0,Somite,D22_2
109793184,Y:9589750-9590648,0.0,Somite,D22_2
109793185,Y:9589776-9590667,0.0,Somite,D22_2


In [ ]:
def get_continous_track(peak_metadata, window_size=100):
    peak_metadata['cont_ATAC'] =  []

    for name, group in peak_metadata.groupby(['cell_type', 'dataset']):
        print(name)
        bw_file = '../results/bam_cell_type/' + name[0] + '/' + name[1] + '.bw'
        bw = pyBigWig.open(bw_file)

        ATAC_track = group.apply(lambda x: bw.values(x.chr[3:], x.start - window_size, x.end + window_size), axis=1)
        peak_metadata[np.logical_and(peak_metadata.cell_type == name[1], peak_metadata.dataset == name[0])] = ATAC_track

get_continous_track(dataset.sequence_ATAC)

In [9]:
import pandas as pd 

cell_types = pd.read_csv('../results/bam_cell_type/D12_1/D12_1_cell_types.tsv', 
                            sep='\t', header=None)
cell_types.columns = ['barcode','cell_type']

cell_types.groupby('cell_type').count()

,barcode
cell_type,
12,555
14,80
Immature neuron,253
Mesenchymal,5610
Myoblast/Myocytes,908
Myogenic progenitor,1802
Neuroblast,396
Neuronal progenitor,5259
Somite,2376
